# Імпортування даних про час кожного кола у сесії в таблиці фактів

На жаль, інформації про час кожного кола для кожного гонщика в сесії не вдалося знайти у датасетах. Проте такі дані надає бібліотека fastf1. Скористаємося її даними та імпортуємо їх у сховище, щоб мати швидкий доступ до них.

Імпортую необхідні бібліотеки

In [44]:
import fastf1
import pandas as pd
from dotenv import dotenv_values
from sqlalchemy import create_engine, text

Завантажуємо змінні оточення з .env файлу для з'єднання зі сховищем

In [45]:
config = dotenv_values()

DB_NAME = config.get('DB_NAME')
DB_USER = config.get('DB_USER')
DB_HOST = config.get('DB_HOST')
DB_PASSWORD = config.get('DB_PASSWORD')
DB_PORT = config.get('DB_PORT')

З'єднуємося з базою даних

In [46]:
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

Створюємо функцію, яка збирає дані про перегони зі сховища від обраного року до нашого часу

In [47]:
def get_races(start_year):
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT * FROM races WHERE EXTRACT(YEAR FROM date) >= {start_year} AND date <= NOW()"))
        data = result.fetchall()
        columns = result.keys()
        return pd.DataFrame(data, columns=columns)

Зберігаємо інформацію про перегони, для яких імпортуємо дані про таймінг кіл з бібліотеки в датафрейм. Дані з бібліотеки датуються починаючи з сезону 2018 і до нашого часу, тож збиратимемо дані в період з 2018-2024.

In [48]:
races_df = get_races(2018)
races_df.head()

,id,season_id,round,date,grand_prix_id,official_name,circuit_id,course_length,laps,distance
0,977,69,1,2018-03-25,4,Formula 1 2018 Rolex Australian Grand Prix,41,5.303,58,307.574
1,978,69,2,2018-04-08,7,Formula 1 2018 Gulf Air Bahrain Grand Prix,8,5.412,57,308.238
2,979,69,3,2018-04-15,12,Formula 1 2018 Heineken Chinese Grand Prix,65,5.451,56,305.066
3,980,69,4,2018-04-29,6,Formula 1 2018 Rolex Azerbaijan Grand Prix,9,6.003,51,306.049
4,981,69,5,2018-05-13,45,Formula 1 Gran Premio de España Emirates 2018,16,4.655,66,307.104


Імпортуємо потрібні нам дані з бібліотеки у відповідні датафрейми

<li>Функція <code>is_sprint_weekend</code>: визначає чи етап чемпіонату містить спринт, на вхід приймається об'єкт Event з бібліотеки fastf1, а повертається значення True або False.</li>
<li>Функція <code>fetch_data_from_session</code>: завантажує дані про конкретну сесію етапу чемпіонату, на вхід приймаються дані про рік та порядковий номер етапу чемпіонату, а також тип сесії, повертається датафрейм з відповідними даними про гонщика, час проходження кола, номер кола та команду.</li>
<li>Функція <code>get_lap_times</code>: завантажує інформацію про таймінги кіл з усіх етапів, які приймаються на вхід функції.</li>

In [49]:
def is_sprint_weekend(event):
    return event.Session3 == 'Sprint' or event.Session4 == 'Sprint'

def fetch_data_from_session(year, round, type):
    session = fastf1.get_session(year, round, type)
    session.load(
        laps = True,
        telemetry = False,
        weather = False,
        messages = False
    )
    drivers_info = []
    for driver in session.drivers:
        driver_info = session.get_driver(driver)
        drivers_info.append({
            'DriverNumber': driver_info['DriverNumber'],
            'FirstName': driver_info['FirstName'],
            'Abbreviation': driver_info['Abbreviation'],
        })

    driver_info_df = pd.DataFrame(drivers_info)

    laps_df = session.laps[['Driver', 'LapTime', 'LapNumber', 'Team', 'DriverNumber']]
    df = laps_df.merge(driver_info_df, on='DriverNumber', how='left')
    return df[['LapTime', 'LapNumber', 'Team', 'FirstName', 'Abbreviation']]

def get_lap_times(races_df):
    sprint_lap_times_df = pd.DataFrame()
    race_lap_times_df = pd.DataFrame()

    for index, row in races_df.iterrows():
        date = row['date']
        round = row['round']
        year = date.year
        race_id = row['id']

        event = fastf1.get_event(year=year, gp=round)
        
        if (is_sprint_weekend(event)):
            sprint_lap_times = fetch_data_from_session(year, round, 'S')
            sprint_lap_times['race_id'] = race_id
            sprint_lap_times_df = pd.concat([sprint_lap_times_df, sprint_lap_times], ignore_index=True)
        
        race_lap_times = fetch_data_from_session(year, round, 'R')
        race_lap_times['race_id'] = race_id
        race_lap_times_df = pd.concat([race_lap_times_df, race_lap_times], ignore_index=True)

    return sprint_lap_times_df, race_lap_times_df            

In [50]:
sprint_lap_times_df, race_lap_times_df = get_lap_times(races_df)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.3.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2018/1/results.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\zagrebelnio\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests_cache\session.py", line 285, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\zagrebelnio\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests_cache\session.py", line 253, in _send_and_cache
    response = super().send(request, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\zagrebelnio\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastf1\req.py", line 125, in send
    lim.limit()
  File "c:\Us

In [51]:
sprint_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id
0,NaT,1.0,Red Bull Racing,Max,VER,1045
1,0 days 00:01:30.623000,2.0,Red Bull Racing,Max,VER,1045
2,0 days 00:01:30.424000,3.0,Red Bull Racing,Max,VER,1045
3,0 days 00:01:30.357000,4.0,Red Bull Racing,Max,VER,1045
4,0 days 00:01:30.478000,5.0,Red Bull Racing,Max,VER,1045


In [52]:
race_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id
0,0 days 00:01:45.060000,1.0,Toro Rosso,Pierre,GAS,977
1,0 days 00:01:33.372000,2.0,Toro Rosso,Pierre,GAS,977
2,0 days 00:01:32.861000,3.0,Toro Rosso,Pierre,GAS,977
3,0 days 00:01:32.184000,4.0,Toro Rosso,Pierre,GAS,977
4,0 days 00:01:32.332000,5.0,Toro Rosso,Pierre,GAS,977


In [53]:
race_lap_times_df.info()

<class 'fastf1.core.Laps'>
RangeIndex: 143438 entries, 0 to 143437
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype          
---  ------        --------------   -----          
 0   LapTime       140266 non-null  timedelta64[ns]
 1   LapNumber     143438 non-null  float64        
 2   Team          143438 non-null  object         
 3   FirstName     143438 non-null  object         
 4   Abbreviation  143438 non-null  object         
 5   race_id       143438 non-null  int64          
dtypes: float64(1), int64(1), object(3), timedelta64[ns](1)
memory usage: 6.6+ MB


In [54]:
sprint_lap_times_df.info()

<class 'fastf1.core.Laps'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   LapTime       4932 non-null   timedelta64[ns]
 1   LapNumber     5365 non-null   float64        
 2   Team          5365 non-null   object         
 3   FirstName     5365 non-null   object         
 4   Abbreviation  5365 non-null   object         
 5   race_id       5365 non-null   int64          
dtypes: float64(1), int64(1), object(3), timedelta64[ns](1)
memory usage: 251.6+ KB


Бачимо що деякі кола не мають дані про таймінг. Це може бути пов'язано з різними причинами. Наприклад, бібліотека не враховує кола заїзду на та виїзду з піт лейну, що є логічним, оскільки ці кола не є репрезентативними відносно гоночного темпу у сесії. Також перегони бувають у режимі віртуального або реального автомобіля безпеки, який сповільнює їх темп під час аварійних інцидентів. Такі кола також можуть не враховуватися.

Тому рядки з датафрейму, що не мають таймінгу кола просто видалимо з датафреймів.

In [55]:
race_lap_times_df = race_lap_times_df.dropna(subset=['LapTime'])
race_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 140266 entries, 0 to 143437
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype          
---  ------        --------------   -----          
 0   LapTime       140266 non-null  timedelta64[ns]
 1   LapNumber     140266 non-null  float64        
 2   Team          140266 non-null  object         
 3   FirstName     140266 non-null  object         
 4   Abbreviation  140266 non-null  object         
 5   race_id       140266 non-null  int64          
dtypes: float64(1), int64(1), object(3), timedelta64[ns](1)
memory usage: 7.5+ MB


In [56]:
sprint_lap_times_df = sprint_lap_times_df.dropna(subset=['LapTime'])
sprint_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 4932 entries, 1 to 5362
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   LapTime       4932 non-null   timedelta64[ns]
 1   LapNumber     4932 non-null   float64        
 2   Team          4932 non-null   object         
 3   FirstName     4932 non-null   object         
 4   Abbreviation  4932 non-null   object         
 5   race_id       4932 non-null   int64          
dtypes: float64(1), int64(1), object(3), timedelta64[ns](1)
memory usage: 269.7+ KB


Для того щоб підігнати дані під сховище потрібно додати новий стовпець lap_time_millis, який буде відображати час у мілісекундах. Для цього використаємо стовпець LapTime (pandas.Timedelta).

In [57]:
race_lap_times_df['lap_time_millis'] = race_lap_times_df['LapTime'].dt.total_seconds() * 1000
race_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id,lap_time_millis
0,0 days 00:01:45.060000,1.0,Toro Rosso,Pierre,GAS,977,105060.0
1,0 days 00:01:33.372000,2.0,Toro Rosso,Pierre,GAS,977,93372.0
2,0 days 00:01:32.861000,3.0,Toro Rosso,Pierre,GAS,977,92861.0
3,0 days 00:01:32.184000,4.0,Toro Rosso,Pierre,GAS,977,92184.0
4,0 days 00:01:32.332000,5.0,Toro Rosso,Pierre,GAS,977,92332.0


In [58]:
sprint_lap_times_df['lap_time_millis'] = sprint_lap_times_df['LapTime'].dt.total_seconds() * 1000
sprint_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id,lap_time_millis
1,0 days 00:01:30.623000,2.0,Red Bull Racing,Max,VER,1045,90623.0
2,0 days 00:01:30.424000,3.0,Red Bull Racing,Max,VER,1045,90424.0
3,0 days 00:01:30.357000,4.0,Red Bull Racing,Max,VER,1045,90357.0
4,0 days 00:01:30.478000,5.0,Red Bull Racing,Max,VER,1045,90478.0
5,0 days 00:01:30.199000,6.0,Red Bull Racing,Max,VER,1045,90199.0


Переведемо стовпці номеру кола та мілісекунд у цілочисельний тип.

In [59]:
race_lap_times_df['LapNumber'] = race_lap_times_df['LapNumber'].astype(int)
race_lap_times_df['lap_time_millis'] = race_lap_times_df['lap_time_millis'].astype(int)

race_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id,lap_time_millis
0,0 days 00:01:45.060000,1,Toro Rosso,Pierre,GAS,977,105060
1,0 days 00:01:33.372000,2,Toro Rosso,Pierre,GAS,977,93372
2,0 days 00:01:32.861000,3,Toro Rosso,Pierre,GAS,977,92861
3,0 days 00:01:32.184000,4,Toro Rosso,Pierre,GAS,977,92184
4,0 days 00:01:32.332000,5,Toro Rosso,Pierre,GAS,977,92332


In [60]:
sprint_lap_times_df['LapNumber'] = sprint_lap_times_df['LapNumber'].astype(int)
sprint_lap_times_df['lap_time_millis'] = sprint_lap_times_df['lap_time_millis'].astype(int)

sprint_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id,lap_time_millis
1,0 days 00:01:30.623000,2,Red Bull Racing,Max,VER,1045,90623
2,0 days 00:01:30.424000,3,Red Bull Racing,Max,VER,1045,90424
3,0 days 00:01:30.357000,4,Red Bull Racing,Max,VER,1045,90357
4,0 days 00:01:30.478000,5,Red Bull Racing,Max,VER,1045,90478
5,0 days 00:01:30.199000,6,Red Bull Racing,Max,VER,1045,90199


Тепер перетворимо стовпець LapTime у формат strinig 00:00.000, який використовується в чемпіонаті.

In [61]:
def timedelta_to_string(td):
    minutes = td.seconds // 60
    seconds = td.seconds % 60
    milliseconds = td.microseconds // 1000
    return '{:02d}:{:02d}.{:03d}'.format(minutes, seconds, milliseconds)

In [62]:
race_lap_times_df['lap_time'] = race_lap_times_df['LapTime'].dt.total_seconds().apply(lambda x: timedelta_to_string(pd.Timedelta(seconds=x)))
race_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id,lap_time_millis,lap_time
0,0 days 00:01:45.060000,1,Toro Rosso,Pierre,GAS,977,105060,01:45.060
1,0 days 00:01:33.372000,2,Toro Rosso,Pierre,GAS,977,93372,01:33.372
2,0 days 00:01:32.861000,3,Toro Rosso,Pierre,GAS,977,92861,01:32.861
3,0 days 00:01:32.184000,4,Toro Rosso,Pierre,GAS,977,92184,01:32.184
4,0 days 00:01:32.332000,5,Toro Rosso,Pierre,GAS,977,92332,01:32.332


In [63]:
sprint_lap_times_df['lap_time'] = sprint_lap_times_df['LapTime'].dt.total_seconds().apply(lambda x: timedelta_to_string(pd.Timedelta(seconds=x)))
sprint_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id,lap_time_millis,lap_time
1,0 days 00:01:30.623000,2,Red Bull Racing,Max,VER,1045,90623,01:30.623
2,0 days 00:01:30.424000,3,Red Bull Racing,Max,VER,1045,90424,01:30.424
3,0 days 00:01:30.357000,4,Red Bull Racing,Max,VER,1045,90357,01:30.357
4,0 days 00:01:30.478000,5,Red Bull Racing,Max,VER,1045,90478,01:30.478
5,0 days 00:01:30.199000,6,Red Bull Racing,Max,VER,1045,90199,01:30.199


Тепер додамо стовпці driver_id згідно з даними зі сховища.
Визначати ідентифікатори будемо за допомогою абревіатур гонщиків. Справа в тому, що одну абревіатуру протягом різних періодів могли мати декілька різних гонщиків, тому доведеться ще враховувати імена гонщиків.

In [64]:
def get_drivers():
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT * FROM drivers"))
        data = result.fetchall()
        columns = result.keys()
        return pd.DataFrame(data, columns=columns)

In [65]:
drivers_df = get_drivers()
drivers_df.head()

,id,name,first_name,last_name,full_name,abbreviation,permanent_number,gender,date_of_birth,nationality_country_id
0,1,Adderly Fong,Adderly,Fong,Adderly Fong Cheun-yue,FON,NaN,MALE,1990-03-02,100
1,2,Adolf Brudes,Adolf,Brudes,Adolf Brudes von Breslau,BRU,NaN,MALE,1899-10-15,84
2,3,Adolfo Schwelm Cruz,Adolfo,Schwelm Cruz,Adolfo Julio Carlos Schwelm Cruz,SCH,NaN,MALE,1923-06-28,11
3,4,Adrián Campos,Adrián,Campos,Adrián Campos Suñer,CAM,NaN,MALE,1960-06-17,210
4,5,Adrian Sutil,Adrian,Sutil,Adrian Sutil,SUT,NaN,MALE,1983-01-11,84


In [66]:
drivers_df['combined_key'] = drivers_df['first_name'] + '_' + drivers_df['abbreviation']
drivers_dict = drivers_df.set_index('combined_key')['id'].to_dict()
race_lap_times_df['combined_key'] = race_lap_times_df['FirstName'] + '_' + race_lap_times_df['Abbreviation']
race_lap_times_df['driver_id'] = pd.NA

race_lap_times_df['driver_id'] = race_lap_times_df.apply(
    lambda row: drivers_dict.get(row['combined_key']) if pd.isnull(row['driver_id']) else row['driver_id'],
    axis=1
)

race_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 140266 entries, 0 to 143437
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype          
---  ------           --------------   -----          
 0   LapTime          140266 non-null  timedelta64[ns]
 1   LapNumber        140266 non-null  int32          
 2   Team             140266 non-null  object         
 3   FirstName        140266 non-null  object         
 4   Abbreviation     140266 non-null  object         
 5   race_id          140266 non-null  int64          
 6   lap_time_millis  140266 non-null  int32          
 7   lap_time         140266 non-null  object         
 8   combined_key     140266 non-null  object         
 9   driver_id        140266 non-null  int64          
dtypes: int32(2), int64(2), object(5), timedelta64[ns](1)
memory usage: 10.7+ MB


In [67]:
drivers_df['combined_key'] = drivers_df['first_name'] + '_' + drivers_df['abbreviation']
drivers_dict = drivers_df.set_index('combined_key')['id'].to_dict()
sprint_lap_times_df['combined_key'] = sprint_lap_times_df['FirstName'] + '_' + sprint_lap_times_df['Abbreviation']
sprint_lap_times_df['driver_id'] = pd.NA

sprint_lap_times_df['driver_id'] = sprint_lap_times_df.apply(
    lambda row: drivers_dict.get(row['combined_key']) if pd.isnull(row['driver_id']) else row['driver_id'],
    axis=1
)

sprint_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 4932 entries, 1 to 5362
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   LapTime          4932 non-null   timedelta64[ns]
 1   LapNumber        4932 non-null   int32          
 2   Team             4932 non-null   object         
 3   FirstName        4932 non-null   object         
 4   Abbreviation     4932 non-null   object         
 5   race_id          4932 non-null   int64          
 6   lap_time_millis  4932 non-null   int32          
 7   lap_time         4932 non-null   object         
 8   combined_key     4932 non-null   object         
 9   driver_id        4932 non-null   int64          
dtypes: int32(2), int64(2), object(5), timedelta64[ns](1)
memory usage: 385.3+ KB


In [68]:
race_lap_times_df.head()

,LapTime,LapNumber,Team,FirstName,Abbreviation,race_id,lap_time_millis,lap_time,combined_key,driver_id
0,0 days 00:01:45.060000,1,Toro Rosso,Pierre,GAS,977,105060,01:45.060,Pierre_GAS,730
1,0 days 00:01:33.372000,2,Toro Rosso,Pierre,GAS,977,93372,01:33.372,Pierre_GAS,730
2,0 days 00:01:32.861000,3,Toro Rosso,Pierre,GAS,977,92861,01:32.861,Pierre_GAS,730
3,0 days 00:01:32.184000,4,Toro Rosso,Pierre,GAS,977,92184,01:32.184,Pierre_GAS,730
4,0 days 00:01:32.332000,5,Toro Rosso,Pierre,GAS,977,92332,01:32.332,Pierre_GAS,730


Тепер додамо стовпці  constructor_id згідно з даними зі сховища.

In [69]:
def get_constructors():
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT * FROM constructors"))
        data = result.fetchall()
        columns = result.keys()
        return pd.DataFrame(data, columns=columns)

In [70]:
constructors_df = get_constructors()
constructors_df.head()

,id,name,full_name,country_id
0,1,Adams,Adams,237
1,2,AFM,Alex von Falkenhausen Motorenbau,84
2,3,AGS,Automobiles Gonfaronnaises Sportives,77
3,4,Alfa Romeo,Alfa Romeo Racing,110
4,5,Alfa Special,Alfa Special,206


In [71]:
constructors_dict = constructors_df.set_index('full_name')['id'].to_dict()
race_lap_times_df['constructor_id'] = pd.NA

race_lap_times_df['constructor_id'] = race_lap_times_df.apply(
    lambda row: constructors_dict.get(row['Team']) if pd.isnull(row['constructor_id']) else row['constructor_id'],
    axis=1
)

race_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 140266 entries, 0 to 143437
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype          
---  ------           --------------   -----          
 0   LapTime          140266 non-null  timedelta64[ns]
 1   LapNumber        140266 non-null  int32          
 2   Team             140266 non-null  object         
 3   FirstName        140266 non-null  object         
 4   Abbreviation     140266 non-null  object         
 5   race_id          140266 non-null  int64          
 6   lap_time_millis  140266 non-null  int32          
 7   lap_time         140266 non-null  object         
 8   combined_key     140266 non-null  object         
 9   driver_id        140266 non-null  int64          
 10  constructor_id   42312 non-null   float64        
dtypes: float64(1), int32(2), int64(2), object(5), timedelta64[ns](1)
memory usage: 11.8+ MB


In [72]:
constructors_dict = constructors_df.set_index('name')['id'].to_dict()
null_constructor_id_df = race_lap_times_df[race_lap_times_df['constructor_id'].isnull()]
null_constructor_id_df['constructor_id'] = null_constructor_id_df.apply(
    lambda row: constructors_dict.get(row['Team']) if pd.isnull(row['constructor_id']) else row['constructor_id'],
    axis=1
)
null_constructor_id_df.info()

<class 'fastf1.core.Laps'>
Index: 97954 entries, 0 to 143437
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   LapTime          97954 non-null  timedelta64[ns]
 1   LapNumber        97954 non-null  int32          
 2   Team             97954 non-null  object         
 3   FirstName        97954 non-null  object         
 4   Abbreviation     97954 non-null  object         
 5   race_id          97954 non-null  int64          
 6   lap_time_millis  97954 non-null  int32          
 7   lap_time         97954 non-null  object         
 8   combined_key     97954 non-null  object         
 9   driver_id        97954 non-null  int64          
 10  constructor_id   97954 non-null  int64          
dtypes: int32(2), int64(3), object(5), timedelta64[ns](1)
memory usage: 8.2+ MB


C:\Users\zagrebelnio\AppData\Local\Temp\ipykernel_13704\3800517809.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_constructor_id_df['constructor_id'] = null_constructor_id_df.apply(


In [73]:
race_lap_times_df = pd.concat([race_lap_times_df, null_constructor_id_df])
race_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 238220 entries, 0 to 143437
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype          
---  ------           --------------   -----          
 0   LapTime          238220 non-null  timedelta64[ns]
 1   LapNumber        238220 non-null  int32          
 2   Team             238220 non-null  object         
 3   FirstName        238220 non-null  object         
 4   Abbreviation     238220 non-null  object         
 5   race_id          238220 non-null  int64          
 6   lap_time_millis  238220 non-null  int32          
 7   lap_time         238220 non-null  object         
 8   combined_key     238220 non-null  object         
 9   driver_id        238220 non-null  int64          
 10  constructor_id   140266 non-null  float64        
dtypes: float64(1), int32(2), int64(2), object(5), timedelta64[ns](1)
memory usage: 20.0+ MB


In [74]:
race_lap_times_df = race_lap_times_df.dropna(subset=['constructor_id'])
race_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 140266 entries, 304 to 143437
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype          
---  ------           --------------   -----          
 0   LapTime          140266 non-null  timedelta64[ns]
 1   LapNumber        140266 non-null  int32          
 2   Team             140266 non-null  object         
 3   FirstName        140266 non-null  object         
 4   Abbreviation     140266 non-null  object         
 5   race_id          140266 non-null  int64          
 6   lap_time_millis  140266 non-null  int32          
 7   lap_time         140266 non-null  object         
 8   combined_key     140266 non-null  object         
 9   driver_id        140266 non-null  int64          
 10  constructor_id   140266 non-null  float64        
dtypes: float64(1), int32(2), int64(2), object(5), timedelta64[ns](1)
memory usage: 11.8+ MB


In [75]:
constructors_dict = constructors_df.set_index('full_name')['id'].to_dict()
sprint_lap_times_df['constructor_id'] = pd.NA

sprint_lap_times_df['constructor_id'] = sprint_lap_times_df.apply(
    lambda row: constructors_dict.get(row['Team']) if pd.isnull(row['constructor_id']) else row['constructor_id'],
    axis=1
)

constructors_dict = constructors_df.set_index('name')['id'].to_dict()
null_constructor_id_df = sprint_lap_times_df[sprint_lap_times_df['constructor_id'].isnull()]
null_constructor_id_df['constructor_id'] = null_constructor_id_df.apply(
    lambda row: constructors_dict.get(row['Team']) if pd.isnull(row['constructor_id']) else row['constructor_id'],
    axis=1
)

sprint_lap_times_df = pd.concat([sprint_lap_times_df, null_constructor_id_df])

sprint_lap_times_df = sprint_lap_times_df.dropna(subset=['constructor_id'])
sprint_lap_times_df.info()

C:\Users\zagrebelnio\AppData\Local\Temp\ipykernel_13704\1021910629.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_constructor_id_df['constructor_id'] = null_constructor_id_df.apply(


<class 'fastf1.core.Laps'>
Index: 4932 entries, 1 to 5324
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   LapTime          4932 non-null   timedelta64[ns]
 1   LapNumber        4932 non-null   int32          
 2   Team             4932 non-null   object         
 3   FirstName        4932 non-null   object         
 4   Abbreviation     4932 non-null   object         
 5   race_id          4932 non-null   int64          
 6   lap_time_millis  4932 non-null   int32          
 7   lap_time         4932 non-null   object         
 8   combined_key     4932 non-null   object         
 9   driver_id        4932 non-null   int64          
 10  constructor_id   4932 non-null   float64        
dtypes: float64(1), int32(2), int64(2), object(5), timedelta64[ns](1)
memory usage: 423.8+ KB


Видалимо непотрібні стовпці, перейменуємо залишені та завантажимо у сховище.

In [76]:
race_lap_times_df = race_lap_times_df[['race_id', 'driver_id', 'constructor_id', 'LapNumber', 'lap_time', 'lap_time_millis']]
sprint_lap_times_df = sprint_lap_times_df[['race_id', 'driver_id', 'constructor_id', 'LapNumber', 'lap_time', 'lap_time_millis']]

In [77]:
race_lap_times_df.rename(columns={'LapNumber': 'lap_number'}, inplace=True)
sprint_lap_times_df.rename(columns={'LapNumber': 'lap_number'}, inplace=True)
race_lap_times_df.head()
sprint_lap_times_df.head()

C:\Users\zagrebelnio\AppData\Local\Temp\ipykernel_13704\3528916211.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sprint_lap_times_df.rename(columns={'LapNumber': 'lap_number'}, inplace=True)


,race_id,driver_id,constructor_id,lap_number,lap_time,lap_time_millis
1,1045,606,141.0,2,01:30.623,90623
2,1045,606,141.0,3,01:30.424,90424
3,1045,606,141.0,4,01:30.357,90357
4,1045,606,141.0,5,01:30.478,90478
5,1045,606,141.0,6,01:30.199,90199


In [78]:
race_lap_times_df['constructor_id'] = race_lap_times_df['constructor_id'].astype('Int64')
sprint_lap_times_df['constructor_id'] = sprint_lap_times_df['constructor_id'].astype('Int64')
race_lap_times_df.head()
sprint_lap_times_df.head()

C:\Users\zagrebelnio\AppData\Local\Temp\ipykernel_13704\2692214647.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sprint_lap_times_df['constructor_id'] = sprint_lap_times_df['constructor_id'].astype('Int64')


,race_id,driver_id,constructor_id,lap_number,lap_time,lap_time_millis
1,1045,606,141,2,01:30.623,90623
2,1045,606,141,3,01:30.424,90424
3,1045,606,141,4,01:30.357,90357
4,1045,606,141,5,01:30.478,90478
5,1045,606,141,6,01:30.199,90199


In [81]:
def insert_data(df, table):
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT COUNT(*) FROM {table}"))
        count = result.scalar()

        if count != 0:
            connection.execute(text(f"DELETE FROM {table}"))
        df.to_sql(table, engine, if_exists='append', index=False)

In [82]:
insert_data(race_lap_times_df, 'races_race_lap_times')
insert_data(sprint_lap_times_df, 'races_sprint_race_lap_times')